In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pickle

path = os.getcwd().split(os.sep + 'GUI')[0]
if path not in sys.path:
    print("not here")
    sys.path.append(path)

from neurolib.models.aln import ALNModel
from neurolib.utils import plotFunctions as plotFunc
from neurolib.utils import costFunctions as cost
import neurolib.dashboard.functions as functions
import neurolib.dashboard.data as data
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2 

#path = os.path.join(os.getcwd(), "plots")

not here


In [2]:
aln = ALNModel()
N = aln.params.N

data.set_parameters(aln)

state_vars = aln.state_vars
init_vars = aln.init_vars

##############################################################
def setinit(init_vars_, model):
    state_vars = model.state_vars
    init_vars = model.init_vars
    for iv in range(len(init_vars)):
        for sv in range(len(state_vars)):
            if state_vars[sv] in init_vars[iv]:
                #print("set init vars ", )
                if model.params[init_vars[iv]].ndim == 2:
                    model.params[init_vars[iv]][0,:] = init_vars_[sv]
                else:
                    model.params[init_vars[iv]][0] = init_vars_[sv]
                    
##############################################################               
def setmaxmincontrol(max_c_c, min_c_c, max_c_r, min_c_r):
    import numpy as np
    
    max_cntrl = np.zeros(( 6 ))
    min_cntrl = np.zeros(( 6 ))
    
    max_cntrl[0] = max_c_c
    min_cntrl[0] = min_c_c
    max_cntrl[1] = max_c_c
    min_cntrl[1] = min_c_c
    max_cntrl[2] = max_c_r
    min_cntrl[2] = min_c_r
    max_cntrl[3] = max_c_r
    min_cntrl[3] = min_c_r
    max_cntrl[4] = max_c_r
    min_cntrl[4] = min_c_r
    max_cntrl[5] = max_c_r
    min_cntrl[5] = min_c_r
            
    return max_cntrl, min_cntrl

In [3]:
##### LOAD BOUNDARIES
data_file = 'bi.pickle'
with open(data_file,'rb') as f:
    load_array= pickle.load(f)
exc = load_array[0]
inh = load_array[1]
print(len(exc))

147


In [4]:
t0_array = np.arange(380., -1, -1.)
t_pen = t0_array / 400.

bestControl_1 = [None] * len(t_pen)
bestState_1 = [None] * len(t_pen)
cost_1 = [None] * len(t_pen)
runtime_1 = [None] * len(t_pen)
grad_1 = [None] * len(t_pen)
phi_1 = [None] * len(t_pen)
costnode_1 = [None] * len(t_pen)
weights_1 = [None] * len(t_pen)

In [5]:
initVars = [None] * len(exc)
target = [None] * len(exc)
cost_uncontrolled = [None] * len(exc)

cgv_list = [None, "HS", "FR", "PR", "CD", "LS", "DY", "WYL", "HZ", None]

In [12]:
dur_pre = 10
dur_post = 10

i = 50

n_pre = int(np.around(dur_pre/aln.params.dt + 1.,1))
n_post = int(np.around(dur_post/aln.params.dt + 1.,1))

tol = 1e-32
start_step = 10.
c_scheme = np.zeros(( 1,1 ))
c_scheme[0,0] = 1.
u_mat = np.identity(1)
u_scheme = np.array([[1.]])

c_var = [ [0], [1], [0,1]]
p_var = [ [0], [0], [0]]

### CURRENTS
cntrl_vars_0 = [0,1]
prec_vars = [0]

max_I = [-3., 3.]
factor_wp = 1. / 0.05
factor_ws = 1.
factor_we = 0.
dur = 400
trans_time = 0.95
    
maxC = [5., -5., 0.18, 0.]

n_dur = int(np.around(dur/aln.params.dt + 1.,1))
max_cntrl, min_cntrl = setmaxmincontrol(maxC[0], maxC[1], maxC[2], maxC[3])

In [7]:
init_file = 'control_3.pickle'
final_file_1 = 'control_tradeoff_3b.pickle'

In [8]:
if os.path.isfile(init_file) :
    print("init file found")
    
    with open(init_file,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1[0] = load_array[0][i]
    bestState_1[0] = load_array[1][i]
    cost_1 [0]= load_array[2][i]
    runtime_1[0] = load_array[3][i]
    grad_1 [0]= load_array[4][i]
    phi_1[0] = load_array[5][i]
    costnode_1[0] = load_array[6][i]
    weights_1[0] = load_array[7][i]

init file found
147


In [9]:
if os.path.isfile(final_file_1) :
    print("final file found")
    
    with open(final_file_1,'rb') as f:
        load_array = pickle.load(f)

    print(len(load_array[0]))

    bestControl_1 = load_array[0]
    bestState_1 = load_array[1]
    cost_1 = load_array[2]
    runtime_1 = load_array[3]
    grad_1 = load_array[4]
    phi_1 = load_array[5]
    costnode_1 = load_array[6]
    weights_1 = load_array[7]

In [13]:
# get initial parameters and target states

print("------- ", i, exc[i], inh[i])
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.

aln.params.duration = 3000.

control0 = aln.getZeroControl()
control0 = functions.step_control(aln, maxI_ = max_I[0])

aln.run(control=control0)

target_rates = np.zeros((2))
target_rates[0] = aln.rates_exc[0,-1] 
target_rates[1] = aln.rates_inh[0,-1]

control0 = functions.step_control(aln, maxI_ = max_I[1])
aln.run(control=control0)

init_state_vars = np.zeros(( len(state_vars) ))
for j in range(len(state_vars)):
    if aln.state[state_vars[j]].size == 1:
        init_state_vars[j] = aln.state[state_vars[j]][0]
    else:
        init_state_vars[j] = aln.state[state_vars[j]][0,-1]

initVars[i] = init_state_vars

aln.params.duration = dur

target[i] = aln.getZeroTarget()
target[i][:,0,:] = target_rates[0]
target[i][:,1,:] = target_rates[1]

-------  50 0.47500000000000014 0.6000000000000003


In [14]:
# get uncontrolled cost

data.set_parameters(aln)
aln.params.ext_exc_current = exc[i] * 5.
aln.params.ext_inh_current = inh[i] * 5.
aln.params.duration = dur

cgv = None

In [15]:
print(np.argmax(np.abs(bestControl_1[0]), axis=2))
print(bestControl_1[0][0,0,1000:2000:40]/5.)

max_percent = (np.argmax(np.abs(bestControl_1[0]), axis=2)[0,0] -100) / 3800
print(max_percent)

[[1676    0    0    0    0    0]]
[-5.90756480e-09 -1.76076093e-08 -5.24797877e-08 -1.68625302e-07
 -5.02591144e-07 -1.49797529e-06 -4.46468526e-06 -1.33068242e-05
 -3.96582966e-05 -1.18169061e-04 -3.51945277e-04 -1.04683941e-03
 -3.09946162e-03 -9.05776962e-03 -2.53447596e-02 -6.33558584e-02
 -1.20731676e-01 -1.44228521e-01 -1.21816229e-01 -8.99879622e-02
 -6.34150416e-02 -4.37259310e-02 -2.97162061e-02 -2.00279271e-02
 -1.31164755e-02]
0.4147368421052632


In [16]:
factor_iteration = 1.

##### initial guess
weight_ = 1.
cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

setinit(initVars[i], aln)

weights_1[0] = cost.getParams()

for j in range(1, 381):

    dur = 400. - np.around(j/10., 1)
    T = 4001 - j
    aln.params.duration = dur

    factor_wp = dur / 20.
    cost.setParams(weight_ * factor_wp, weight_ * factor_we, weight_ * factor_ws)

    print('-----', j)
    setinit(initVars[i], aln)

    max_it = 3.
    if j > 170:
        max_it = 40.
    elif j > 140:
        max_it = 30.
    elif j > 100.:
        max_it = 20.
    
    if type(bestControl_1[j]) == type(None):
        total_shift = round(j * max_percent * 10)
        prev_shift = round( (j-1) * max_percent * 10)
        shift_ind = total_shift - prev_shift
        #print(total_shift, prev_shift, shift_ind)
        control0 = np.zeros(( bestControl_1[j-1][:,:,100:-101].shape ))
        control0[:,:,:-shift_ind] = 1. * bestControl_1[j-1][:,:,100+shift_ind:-101]
    else:
        if bestControl_1[j].shape[2] == T+200:
            control0 = bestControl_1[j][:,:,100:-100]
        else:
            control0 = bestControl_1[j][:,:,100:-(100+j)]

    bestControl_1[j], bestState_1[j], cost_1[j], runtime_1[j], grad_1[j], phi_1[j], costnode_1[j] = aln.A1(
        control0, target[i], c_scheme, u_mat, u_scheme, max_iteration_ = max_it, tolerance_ = tol,
        startStep_ = start_step, max_control_ = max_cntrl, min_control_ = min_cntrl, t_sim_ = dur,
        t_sim_pre_ = dur_pre, t_sim_post_ = dur_post, CGVar = cgv, control_variables_ = [0,1],
        prec_variables_ = prec_vars, transition_time_ = t_pen[j])

    print(costnode_1[j][0][0][0], costnode_1[j][2][0][1])

    print(costnode_1[0][0][0][0], costnode_1[0][2][0][1])
        
    with open(final_file_1,'wb') as f:
        pickle.dump([bestControl_1, bestState_1, cost_1, runtime_1, grad_1, phi_1,
                costnode_1, weights_1], f)

set cost params:  20.0 0.0 1.0
set cost params:  19.994999999999997 0.0 1.0
----- 1
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.6571110999246943
Gradient descend method:  None
RUN  1 , total integrated cost =  2.657110955019301
RUN  2 , total integrated cost =  2.6571109549882124
RUN  3 , total integrated cost =  2.657110954985942
RUN  3 , total integrated cost =  2.657110954985942
Improved over  3  iterations in  32.93494094400012  seconds by  5.454749413047466e-06  percent.


ERROR:root:Problem in initial value trasfer


Problem in initial value trasfer:  Vmean_exc -73.76947335960494 -73.77297874864682
1.2622804743135636e-09 0.0
8.241242234202802e-09 0.0
set cost params:  19.990000000000002 0.0 1.0
----- 2
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.657110958499663
Gradient descend method:  None
RUN  1 , total integrated cost =  2.6571109583318155
RUN  2 , total integrated cost =  2.657110958330638


ERROR:root:Problem in initial value trasfer


RUN  3 , total integrated cost =  2.6571109582887247
RUN  3 , total integrated cost =  2.6571109582887247
Improved over  3  iterations in  2.9411497250002867  seconds by  7.938638191262726e-09  percent.
Problem in initial value trasfer:  Vmean_exc -73.7881054060075 -73.7915243661571
1.3678521275063602e-09 0.0
8.241242234202802e-09 0.0
set cost params:  19.985 0.0 1.0
----- 3
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  2.657110962829275
Gradient descend method:  None
RUN  1 , total integrated cost =  2.657110962331617
RUN  2 , total integrated cost =  2.6571109623253752


...

RUN  40 , total integrated cost =  2.973530012668548
RUN  40 , total integrated cost =  2.973530012668548
Improved over  40  iterations in  27.43985478799732  seconds by  0.9896040658266259  percent.
Problem in initial value trasfer:  Vmean_exc -74.526872469918 -74.52687248097862
0.0014996407912763062 0.0
8.241242234202802e-09 0.0
set cost params:  18.305 0.0 1.0
----- 339
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3.0301490100794815
Gradient descend method:  None
RUN  1 , total integrated cost =  2.9975468341443987
RUN  2 , total integrated cost =  2.9973282375195365
RUN  3 , total integrated cost =  2.997133893880013
RUN  4 , total integrated cost =  2.9969684242203516
RUN  5 , total integrated cost =  2.996807991404341
RUN  6 , total integrated cost =  2.9966515659713835
RUN  7 , total integrated cost =  2.996493901928215
RUN  8 , total integrated cost =  2.9962758940876686
RUN  9 , total integrated cost =  2.996058097347625
RUN  10 , total integrated co

ERROR:root:Problem in initial value trasfer


RUN  40 , total integrated cost =  2.992688179104033
RUN  40 , total integrated cost =  2.992688179104033
Improved over  40  iterations in  27.35944120800559  seconds by  1.2362702576948692  percent.
Problem in initial value trasfer:  Vmean_exc -74.52687254767736 -74.52687255837893
0.001578850505840111 0.0
8.241242234202802e-09 0.0
set cost params:  18.3 0.0 1.0
----- 340
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3.0455159528155664
Gradient descend method:  None
RUN  1 , total integrated cost =  3.017461464578755
RUN  2 , total integrated cost =  3.0173533523484846
RUN  3 , total integrated cost =  3.017090359245609
RUN  4 , total integrated cost =  3.016894500896113
RUN  5 , total integrated cost =  3.0166345287084937
RUN  6 , total integrated cost =  3.016440890674012
RUN  7 , total integrated cost =  3.0162241776418215
RUN  8 , total integrated cost =  3.0160522235811653
RUN  9 , total integrated cost =  3.015798161025297
RUN  10 , total integrated cost

ERROR:root:Problem in initial value trasfer


Control only changes marginally.
RUN  21 , total integrated cost =  3.0136753069062046
Improved over  21  iterations in  15.189136886001506  seconds by  1.0454926653700625  percent.
Problem in initial value trasfer:  Vmean_exc -74.52687262708427 -74.52687263741916
0.0016425935950423096 0.0
8.241242234202802e-09 0.0
set cost params:  18.294999999999998 0.0 1.0
----- 341
interpolate adjoint :  True True True
RUN  0 , total integrated cost =  3.0794633521972212
Gradient descend method:  None
RUN  1 , total integrated cost =  3.04308554704879
RUN  2 , total integrated cost =  3.0428750955245407
RUN  3 , total integrated cost =  3.042700574093139
RUN  4 , total integrated cost =  3.0425123045989997
RUN  5 , total integrated cost =  3.042360575259303
RUN  6 , total integrated cost =  3.04221225348678
RUN  7 , total integrated cost =  3.042073687509343
RUN  8 , total integrated cost =  3.041949566558328
RUN  9 , total integrated cost =  3.041819938877471
RUN  10 , total integrated cost =  3.0